In [0]:
# Setup :)
import numpy as np
from keras.models import Model, Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers import Flatten, Conv3D, Dense, Conv1D, Input
from keras.engine.input_layer import Input
from keras.losses import logcosh

# Uses Keras functional API: https://keras.io/getting-started/functional-api-guide/

In [0]:
# TODO: LOSS FUNCTION

In [57]:
########################## ENCODER NETWORK #################################
# 5 layers
# starts with 32x32x32 input, ends with Fully Connected layer (flattened 7x7x7 output of 4th convolutional layer)

# ???
# Glorot initialization is deafault (glorot_uniform) - Keras has glorot_normal 
# and glorot_uniform, paper is unclear which should be used

# Input is 32x32x32 point cloud
inputs = Input(shape=(32, 32, 32, 1))

# First convolutional layer: outputs 30x30x30
encode_c1 = Conv3D(8, kernel_size=3, activation='elu', padding='valid',
              data_format='channels_last')(inputs)
encode_b1 = BatchNormalization()(encode_c1)

# Second convolutional layer: outputs 15x15x15 (downsamples via striding)
encode_c2 = Conv3D(16, kernel_size=3, activation='elu', padding='same',
              strides=(2, 2, 2))(encode_b1)
encode_b2 = BatchNormalization()(encode_c2)

# Third convolutional layer: outputs 13x13x13
encode_c3 = Conv3D(32, kernel_size=3, activation='elu', padding='valid')(encode_b2)
encode_b3 = BatchNormalization()(encode_c3)

# Fourth convolutional layer: outputs 7x7x7 (downsamples via striding)
encode_c4 = Conv3D(64, kernel_size=3, activation='elu', padding='same', 
              strides=(2, 2, 2))(encode_b3)
encode_b4 = BatchNormalization()(encode_c4)

# Fifth layer, fully connected: outputs 343
encode_f5 = Flatten()(encode_b4)

encoder = Model(inputs=inputs, outputs=encode_f5)

# Structure/info about encoder
encoder.compile(optimizer='adam', loss='logcosh', metrics=['accuracy'])
encoder.summary()

# Testing encoder on dummy data
dummy_data = np.random.rand(100, 32, 32, 32, 1)
# Labels don't matter for us, we only care about the model's ouptut, output must be in the shape of final layer!!!
dumb_labels = np.random.rand(100, 21952)
  
encoder.fit(dummy_data, dumb_labels, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 32, 32, 32, 1)     0         
_________________________________________________________________
conv3d_96 (Conv3D)           (None, 30, 30, 30, 8)     224       
_________________________________________________________________
batch_normalization_89 (Batc (None, 30, 30, 30, 8)     32        
_________________________________________________________________
conv3d_97 (Conv3D)           (None, 15, 15, 15, 16)    3472      
_________________________________________________________________
batch_normalization_90 (Batc (None, 15, 15, 15, 16)    64        
_________________________________________________________________
conv3d_98 (Conv3D)           (None, 13, 13, 13, 32)    13856     
_________________________________________________________________
batch_normalization_91 (Batc (None, 13, 13, 13, 32)    128       
__________

In [0]:
########################## LATENT LAYER #################################

# Sixth layer - LATENT LAYER: outputs 100
latent = Dense(100, use_bias=True, activation='elu')(encode_f5)

In [0]:
########################## DECODER NETWORK ################################# 

# Fully connected layer that takes latent space as input
vae.add(Dense(343, use_bias=True, activation='elu'))

# Convolution layer that convolutes fully connected layer into 7x7x7
vae.add(Conv1D(64, kernel_size=3, activation='elu', padding='same'))

In [0]:
# OLD ENCODER NETWORK IMPLEMENTATION: (doesn't use functional keras)
# vae = Sequential()
# # First convolutional layer
# vae.add(Conv3D(8, kernel_size=3, activation='elu', padding='valid', 
#                    data_format='channels_last', input_shape=(32, 32, 32, 1))) 
# vae.add(BatchNormalization())

# # Second convolutional layer, downsampling through strided convolutions 
# # (default striding is 1x1x1, so up this to 2x2x2)
# vae.add(Conv3D(16, kernel_size=3, activation='elu', padding='same', strides=(2, 2, 2))) 
# vae.add(BatchNormalization())

# # # Third convolutional layer
# vae.add(Conv3D(32, kernel_size=3, activation='elu', padding='valid'))
# vae.add(BatchNormalization())

# # # Fourth convolutional layer, downsampling through strided convolutions
# vae.add(Conv3D(64, kernel_size=3, activation='elu', padding='same', strides=(2, 2, 2)))
# vae.add(BatchNormalization())

# # # Fully connected layer and latent layer
# # # Fully connected: Dense layer that takes in flattened output of previous layer 
# # # Latent layer: output of fully connected layer is 100 dimensional latent space
# vae.add(Flatten())
# vae.add(Dense(100, use_bias=True, activation='elu'))

# vae.compile(optimizer='adam', loss='logcosh', metrics=['accuracy'])
# vae.summary()